# Convert RAW Files to TIFF Files

This script automates the process of converting RAW image files from the SONY RX10 IV camera to TIFF format. It works by iterating over temperature folders (e.g., 650, 700, ..., 1200), processing each RAW file within those folders, and saving the converted TIFF files to a corresponding results folder.

### Steps:
1. Iterates over folders named with numeric temperature values (e.g., 650, 700, ..., 1200).
2. Lists all `.ARW` (RAW) files in each folder and extracts the metadata.
3. Converts each `.ARW` file to a `.TIFF` file.
4. Saves the converted TIFF files into the corresponding `results_folder`.

### Versions:
- **Version 0.1**: 12/22/2023  
  Initial release.

- **Version 0.2**: 01/05/2023  
  Added background conversion from RAW to TIFF (not always required).

- **Version 0.3**: 02/10/2024  
  - Included `-q 1` in DCRAW arguments for Variable Number of Gradients (VNG) interpolation.
  - Checks only numeric string folders for iteration.
  - Saves the resulting `.tiff` files into a `results_folder`.

### Requirements:
- DCRAW for RAW to TIFF conversion (install separately).
- Python environment with necessary libraries (e.g., `os`, `glob`).

### Acknowledgments:

This script uses [DCRAW](https://www.dechifro.org/dcraw/) for converting RAW files to TIFF.

In [1]:
import argparse
import subprocess as sp
import os
from os import listdir
from os.path import isfile, join,exists, isdir
import glob
from glob import iglob
import sys
import pyexiv2
from datetime import datetime

today_date = datetime.now().strftime('%m_%d_%Y')

In [2]:
# DCRAW path & arguments for conversion
main_path = os.getcwd()
dcraw_path = main_path
dcraw_args = " -v -T -4 -q 1"

In [3]:
# Directory of images & results
main_folder = os.path.join(main_path , 'img')

results_folder = os.path.join(main_folder, 'Results_DCRAW_' + today_date)
os.makedirs(results_folder, exist_ok=True)

# To check if directory is numeric (temperature)
def is_numeric(s):
    return s.isdigit()

In [7]:
# Iterate through subdirectories in the main folder
for subfolder in iglob(os.path.join(main_folder, '*'), recursive=False):
    if os.path.isdir(subfolder) and is_numeric(os.path.basename(subfolder)):
        print(f"\nProcessing files in folder: {os.path.basename(subfolder)}")
        
        # List all .ARW files in the subfolder
        arw_files = [f for f in iglob(os.path.join(subfolder, '*.ARW')) if os.path.isfile(f)]

        for idx, arw_file in enumerate(arw_files):
            print(f"Processing file: {os.path.basename(arw_file)}")

            # Run DCRAW to convert the RAW file to TIFF
            sp.call(dcraw_path + "\\dcraw " + dcraw_args + ' "' + arw_file + '"')

            # Get the corresponding .JPG file and create the TIFF file path
            jpg_file = os.path.splitext(arw_file)[0] + ".JPG"
            tiff_file = os.path.join(subfolder, os.path.basename(os.path.splitext(arw_file)[0] + ".tiff").replace("\\", "\\\\"))

            # Move the result to the Results_DCRAW folder
            result_path = os.path.join(results_folder, os.path.basename(subfolder), os.path.basename(tiff_file))
            os.makedirs(os.path.dirname(result_path), exist_ok=True)
            os.rename(tiff_file, result_path)

            # Progress display
            progress_step = 20
            progress = (idx + 1) * progress_step // len(arw_files)
            sys.stdout.write('\r')
            sys.stdout.write("[%-20s] %d%%" % ('=' * progress, 100 * (idx + 1) // len(arw_files)))
            sys.stdout.flush()

            # Transfer metadata from JPG to TIFF if JPG exists
            if os.path.exists(jpg_file):
                with pyexiv2.Image(jpg_file) as img:
                    jpg_data = img.read_exif()

                with pyexiv2.Image(result_path) as img:
                    data = img.read_exif()
                    img.modify_exif({'Exif.Photo.ExposureTime': jpg_data["Exif.Photo.ExposureTime"],
                                     'Exif.Photo.ISOSpeedRatings': jpg_data["Exif.Photo.ISOSpeedRatings"],
                                     'Exif.Photo.FNumber': jpg_data["Exif.Photo.FNumber"]})

        print(f"\nConversion completed for folder: {subfolder}")


Processing files in folder: 650
Processing file: Cal00372.ARW
[======              ] 33%Processing file: Cal00373.ARW
[=============       ] 66%Processing file: Cal00374.ARW
[====================] 100%
Conversion completed for folder: C:\Users\Jorge\OneDrive - Worcester Polytechnic Institute (wpi.edu)\GitHub\color-ratio-pyrometry\img\650

Processing files in folder: 700
Processing file: Cal00412.ARW
[======              ] 33%Processing file: Cal00413.ARW
[=============       ] 66%Processing file: Cal00414.ARW
[====================] 100%
Conversion completed for folder: C:\Users\Jorge\OneDrive - Worcester Polytechnic Institute (wpi.edu)\GitHub\color-ratio-pyrometry\img\700
